In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_user = "myuser"         # Replace with your database username
db_password = "mypassword" # Replace with your database password
db_host = "localhost"            # Replace with the host, e.g., 'localhost' or Docker's IP
db_port = 5432                   # The port PostgreSQL is exposed on
db_name = "mydatabase"         # Replace with your database name

# Connection string
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection by querying the database
try:
    with engine.connect() as connection:
        print("Connection successful!")
        # Example query: List all tables
        result = connection.execute("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public';")
        tables = result.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(table[0])
except Exception as e:
    print(f"Failed to connect to the database: {e}")

Connection successful!
Failed to connect to the database: Not an executable object: "SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public';"


In [2]:
# Query to fetch data
query = "SELECT * FROM addresses LIMIT 10;"  # Replace 'your_table_name' with an actual table name

# Load query results into a DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df.head())

  business_id              type        street  post_code post_office_box  \
0   0782323-7    Postal address    Ritarikatu      170.0            None   
1   0782324-5  Visiting address       Patotie     1600.0            None   
2   0782324-5    Postal address       Patotie     1600.0            None   
3   0782326-1    Postal address  Itämerenkatu      180.0            None   
4   0782328-8    Postal address     JOKIRANTA     1400.0            None   

  building_number entrance  apartment_number apartment_id_suffix  \
0               7        A               2.0                None   
1               2     None               NaN                None   
2               2     None               NaN                None   
3               8        E              96.0                None   
4              18        A               NaN                None   

              co  country free_address_line registration_date  source  
0           None  Finland              None        2023-03-27 

In [3]:
# Query to fetch data
query ="""
SELECT COUNT(*)
FROM companies
WHERE status = 'Valid' ;
"""
# Load query results into a DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df)

    count
0  437516


In [4]:

# Query to fetch rows where type_code_set contains "TOIMI" but not "TOIMI3"
query = """
SELECT *
FROM main_business_lines
WHERE type_code_set LIKE 'TOIMI3';
"""
# Load query results into a DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df)

       business_id                                               type  \
0        2761680-9                       Letting of other real estate   
1        2761688-4  Wholesale of diagnostic instruments and medica...   
2        2761689-2                       Letting of other real estate   
3        2761690-5                       Letting of other real estate   
4        2761691-3  Operation of dwellings and residential real es...   
...            ...                                                ...   
872367   3274626-1          Operation of paid parking lots or garages   
872368   3274628-8                                        Restaurants   
872369   3274629-6                 Wholesale of perfume and cosmetics   
872370   3274637-6  Photographic studio and other photographic act...   
872371   3274638-4          Direct and outdoor advertising activities   

       type_code_set registration_date              source  
0             TOIMI3        2016-10-01  Tax Administration  
1

In [5]:
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

# Execute the query and load results into a DataFrame
tables_df = pd.read_sql(query, engine)

# Display the table names
print("Tables in the database:")
print(tables_df)

Tables in the database:
                       table_name
0                       companies
1       company_form_descriptions
2                           names
3                       addresses
4             main_business_lines
5      business_line_descriptions
6              registered_entries
7                    post_offices
8   registered_entry_descriptions
9              company_situations
10                  company_forms


In [15]:
query = """
SELECT c.business_id, n.name AS company_name, c.registration_date, c.status, po.city
FROM companies c
JOIN post_offices po ON c.business_id = po.business_id
JOIN names n ON c.business_id = n.business_id
WHERE po.city = 'NOKIA'
  AND c.status = 'Valid'; 
"""

# Execute the query and load results into a DataFrame
tables_df = pd.read_sql(query, engine)

# Display the table names
print(tables_df)

      business_id                                   company_name  \
0       2916425-2                         KJ-Kiinteistöhuolto OY   
1       2916425-2                         KJ-Kiinteistöhuolto OY   
2       2919081-3  Tili- ja konsultointipalvelu KOTI Avoin Yhtiö   
3       2919081-3  Tili- ja konsultointipalvelu KOTI Avoin Yhtiö   
4       2919185-9                         Pirkan Eläinlääkäri Oy   
...           ...                                            ...   
12347   3132728-4                                   Kultalasi Oy   
12348   3132728-4                                   Kultalasi Oy   
12349   3132728-4                                  Niittylä Home   
12350   3132728-4                                  Niittylä Home   
12351   3132528-9                Kiinteistö Oy Harjuniityn Halli   

      registration_date status   city  
0            2018-05-29  Valid  NOKIA  
1            2018-05-29  Valid  NOKIA  
2            2018-06-26  Valid  NOKIA  
3            2018-06-26